In [1]:
import pandas as pd
import re             #re正则表达式 

一、读取数据

In [2]:
data=pd.read_excel('螺蛳粉店铺数据.xlsx')
print(data.shape)
print(data.info())

(4404, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 5 columns):
goods_name      4404 non-null object
shop_name       4404 non-null object
price           4404 non-null float64
purchase_num    4404 non-null object
location        4404 non-null object
dtypes: float64(1), object(4)
memory usage: 172.1+ KB
None


In [3]:
print(data.head())

goods_name shop_name  price  purchase_num location
0              商用意大利面通心粉螺旋面粉30斤意面螺蛳粉   尚膳食品专营店  100.0     ['27人付款']    山东 济南
1     预售 李子柒柳州螺蛳粉广西特产正宗螺丝粉方便面米线螺狮粉3袋    李子柒旗舰店   39.7   ['65万+人付款']    浙江 嘉兴
2  嘻螺会柳州正宗螺蛳粉300gx5包广西特产螺狮粉酸辣螺丝粉速食米线  嘻螺会食品旗舰店   49.9  ['9.5万+人付款']    广西 柳州
3  螺霸王螺蛳粉280g*10包装礼盒正宗广西柳州螺狮粉特产螺丝粉整箱   丝皇食品专营店  115.0   ['2059人付款']    广西 柳州
4    预售李子柒柳州螺蛳粉广西特产正宗螺丝粉方便面速食米线11袋礼盒    李子柒旗舰店  139.0  ['5500+人付款']    浙江 嘉兴


二、数据清洗
1.数据去重
2.去除购买人数为空的数据
3.新增purchase_num_new字段：purchase_num由字符串转换成数值型
4.新增sales字段：sales=price*purchase_num
5.新增province和city字段：location里提取省份和城市
6.新增price_cut字段：增加price分箱数据
7.新增shop_type字段：shop_name可以分为官方店和非官方店
8.新增weight字段：goods_name提取重量信息

In [4]:
#1.数据去重+2.去除购买人数为空的数据
data.drop_duplicates(inplace=True)  
data=data[data['purchase_num'].str.contains('人')]
print(data.shape)

(4383, 5)


In [5]:
#3.新增purchase_num_new字段：purchase_num由字符串转换成数值型
data['purchase_num']=[re.sub('\[\'|\'\]','',i) for i in data['purchase_num']]
data['num']=[''.join(re.findall('\d+\.{0,1}\d*',i)) for i in data['purchase_num']]
data['num']=data['num'].astype(float)
data['unit']=[''.join(re.findall('万',i)) for i in data['purchase_num']]
data['unit']=data['unit'].apply(lambda x: 10000 if x=='万' else 1)
data['purchase_num_new']=data['num']*data['unit']
data.drop(['num', 'unit'], axis=1, inplace=True) # 删除多余的列

#4.新增sales字段：sales=price*purchase_num 
data['sales']=data['purchase_num_new']*data['price']

#5.新增province和city字段：location里提取省份和城市 
data['province']=data['location'].str.split(' ').apply(lambda x: x[0])
data['city']=data['location'].str.split(' ').apply(lambda x: x[1] if len(x)>1 else x[0])

In [6]:
#6.新增price_cut字段：增加price分箱数据
print(data['price'].max(),data['price'].min())

1100.0 4.9


In [7]:
def tranform_price(x):
    if x <= 20:
        return '0~20'
    elif x <= 50:
        return '20~50'
    elif x <= 100:
        return '50~100'
    elif x <= 200:
        return '100~200'
    else:
        return '200~1200'    
data['price_cut'] = data['price'].apply(lambda x: tranform_price(x)) 

In [8]:
#7.新增shop_type字段：shop_name可以分为官方店和非官方店
data['shop_type']=data['shop_name'].str.contains('旗舰')|data['shop_name'].str.contains('天猫')|data['shop_name'].str.contains('专营')|data['shop_name'].str.contains('官方')
data['shop_type']=data['shop_type'].apply(lambda x: '官方店' if x==True else '非官方店')

#8.新增weight字段：goods_name提取重量信息
data['weight']=[''.join(re.findall('\d+\.{0,1}\d*g|\d+\.{0,1}\d*G',i)) for i in data['goods_name']]
data['weight']=data['weight'].str.lower()

print(data.head())

goods_name shop_name  price purchase_num location  \
0              商用意大利面通心粉螺旋面粉30斤意面螺蛳粉   尚膳食品专营店  100.0        27人付款    山东 济南   
1     预售 李子柒柳州螺蛳粉广西特产正宗螺丝粉方便面米线螺狮粉3袋    李子柒旗舰店   39.7      65万+人付款    浙江 嘉兴   
2  嘻螺会柳州正宗螺蛳粉300gx5包广西特产螺狮粉酸辣螺丝粉速食米线  嘻螺会食品旗舰店   49.9     9.5万+人付款    广西 柳州   
3  螺霸王螺蛳粉280g*10包装礼盒正宗广西柳州螺狮粉特产螺丝粉整箱   丝皇食品专营店  115.0      2059人付款    广西 柳州   
4    预售李子柒柳州螺蛳粉广西特产正宗螺丝粉方便面速食米线11袋礼盒    李子柒旗舰店  139.0     5500+人付款    浙江 嘉兴   

   purchase_num_new       sales province city price_cut shop_type weight  
0              27.0      2700.0       山东   济南    50~100       官方店         
1          650000.0  25805000.0       浙江   嘉兴     20~50       官方店         
2           95000.0   4740500.0       广西   柳州     20~50       官方店   300g  
3            2059.0    236785.0       广西   柳州   100~200       官方店   280g  
4            5500.0    764500.0       浙江   嘉兴   100~200       官方店         


三.数据可视化 
1.goods_name字段的词云分析
2.不同价格销量占比
3.不同类型店铺的数量占比
4.不同类型店铺的销量占比
5.螺蛳粉销量店铺排名top10
6.各省份店铺数量排名top10
7.各省份螺蛳粉销售收入分布图
8.各省份螺蛳粉销量分布图
9.广西省各市螺蛳粉销量分布图

In [9]:
#词云分析

import jieba          
import jieba.analyse 

goods_data=data['goods_name'].str.cat(sep='。') 

#增加词库
jieba.add_word('螺蛳粉')  

#读取停用词
stop_words=[]
with open('chineseStopWords.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        stop_words.append(line.strip())     

# 评论字段分词处理
word_num = jieba.analyse.extract_tags(goods_data,
                                      topK=100,
                                      withWeight=True,
                                      allowPOS=())

# 删除停用词
word_num_selected = []
for i in word_num:
    if i[0] not in stop_words:
        word_num_selected.append(i)

#分词数据框
key_words = pd.DataFrame(word_num_selected, columns=['word','num'])
key_words.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\pr\AppData\Local\Temp\jieba.cache
Loading model cost 1.068 seconds.
Prefix dict has been built successfully.


,word,num
0,螺蛳粉,1.136864
1,柳州,0.670196
2,正宗,0.491532
3,螺丝,0.480384
4,速食,0.460393


In [10]:
# 词云图
#结论：从词云图上可看出，商品标题中最多的字眼是螺蛳粉/广西/柳州/正宗/特产/速食/包邮/袋装/米线/米粉/酸辣粉/方便面等

from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType
from pyecharts import options as opts

word1 = WordCloud(init_opts=opts.InitOpts(width='1350px', height='750px'))
word1.add("", [*zip(key_words.word, key_words.num)],
          word_size_range=[20, 200],
          shape=SymbolType.DIAMOND)
word1.set_global_opts(title_opts=opts.TitleOpts('商品标题词云图'),
                      toolbox_opts=opts.ToolboxOpts())

In [11]:
#重量分布
#结论：多数螺蛳粉每份重量都集中在300g左右，14%的螺蛳粉每份重量为300g
weight_percent=data['weight'].value_counts()/data['weight'].count()
weight_percent.head(5)

        0.460643
300g    0.143281
320g    0.066849
280g    0.057495
400g    0.054757
Name: weight, dtype: float64

In [12]:
#价格区间分布
#结论：44.5%的商品价格在20-50元区间内，30.9%的商品价格在50-100元区间内
price_percent = data.price_cut.value_counts()/data.price_cut.count()
price_percent

20~50       0.445357
50~100      0.308921
100~200     0.178873
0~20        0.061373
200~1200    0.005476
Name: price_cut, dtype: float64

In [13]:
from pyecharts.charts import Pie
from pyecharts import options as opts

# 定义饼图函数
def pie_self(data_pair,series_name,title):
    c= (Pie(init_opts=opts.InitOpts(width='1350px', height='750px'))
        .add( 
        series_name=series_name,
        radius=["35%", "55%"],
        data_pair=data_pair,
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],},
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,},
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,},
            },),)
        .set_global_opts(legend_opts=opts.LegendOpts(pos_left="left", pos_top='30%', orient="vertical"), 
                     toolbox_opts=opts.ToolboxOpts(),
                     title_opts=opts.TitleOpts(title=title))
        .set_series_opts(tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"))
    )
    return c

In [14]:
#不同价格销量占比
#从饼图中可以看出，20~50的价格区间产生了75%的销量，说明大多数消费者购买螺蛳粉时更偏爱20-50的价格。


#计算不同价格的销量
price_cut_num = data.groupby('price_cut')['purchase_num_new'].sum() 
data_pair = [list(z) for z in zip(price_cut_num.index, price_cut_num.values)]

#饼图
pie1=pie_self(data_pair,"销量",'不同价格销量占比')

In [15]:
#不同类型店铺的数量占比
#结论：从饼图可以看出，官方店铺仅占26.88%。

#自定义函数，计算各类型店铺的去重数量
def countShop(dataframe):
    a={}
    for index,row in dataframe.iterrows():
        outer = row.iloc[0]
        inner = row.iloc[1]
        if outer not in a:
            a[outer] = {}
        if inner not in a[outer]:
            a[outer][inner] = 1
        else:
            a[outer][inner] += 1
    return {k: len(v) for k,v in a.items()}

# 计算不同类型店铺的数量
shop_type_count = pd.DataFrame(sorted(countShop(data[['shop_type','shop_name']]).items(), key = lambda x:x[1],reverse=True)).set_index(0,drop=True).loc[:,1]
data_pair = [list(z) for z in zip(shop_type_count.index, shop_type_count.values.astype(float))]

#饼图
pie2=pie_self(data_pair,"店铺数量",'不同类型店铺的数量占比')

In [16]:
#不同类型店铺的销量占比
#从饼图中可以看出，官方店铺产生了83%的销量，虽然官方店铺并不多，但是大多数消费者购买螺蛳粉时更偏爱到官方店购买，因为质量有保证。

#计算不同类型店铺的销量
shop_type_num = data.groupby('shop_type')['purchase_num_new'].sum() 
data_pair = [list(z) for z in zip(shop_type_num.index, shop_type_num.values)]

#饼图
pie3=pie_self(data_pair,"销量",'不同类型店铺的销量占比')

In [17]:
#螺蛳粉销量店铺排名top10
#结论：从条形图可以看出，在销量top10的店铺，李子柒旗舰店、好欢螺旗舰店、嘻螺会鼎蓉鲜专卖店、天猫超市分别位居前四位。李子柒旗舰店、好欢螺旗舰店的销量均超过50万人次购买，而嘻螺会鼎蓉鲜专卖店销量才刚刚过20万，说明消费者对于李子柒和好欢螺两个品牌有较强的消费偏好，说明它们可能具有品牌搜索护城河。

from pyecharts.charts import Bar
from pyecharts import options as opts 

# 计算top10店铺
shop_top10 = data.groupby('shop_name')['purchase_num_new'].sum().sort_values(ascending=False).head(10)

# 绘制条形图
bar1 = Bar(init_opts=opts.InitOpts(width='1350px', height='750px')) 
bar1.add_xaxis(shop_top10.index.tolist())
bar1.add_yaxis('螺蛳粉销量', shop_top10.values.tolist()) 
bar1.set_global_opts(title_opts=opts.TitleOpts(title='螺蛳粉销量店铺排名Top10'),
                     xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-10)),
                     visualmap_opts=opts.VisualMapOpts(max_=shop_top10.values.max())) 

In [18]:
#各省份店铺数量排名Top10
#结论：从条形图可以看出，在店铺数量top10的省份中，广西、广东、上海分别位居前三位。其中广西的店铺有849个，其余省份的店铺数量均不足100个，这个结果无可厚非，因为螺蛳粉是属于广西的特产。


# 计算top10省份
province_top10 = pd.DataFrame(sorted(countShop(data[['province','shop_name']]).items(), key = lambda x:x[1],reverse=True)).set_index(0,drop=True).loc[:,1].head(10)

# 绘制条形图
bar2 = Bar(init_opts=opts.InitOpts(width='1350px', height='750px')) 
bar2.add_xaxis(province_top10.index.tolist())
bar2.add_yaxis('店铺数量', province_top10.values.tolist()) 
bar2.set_global_opts(title_opts=opts.TitleOpts(title='店铺数量各省份排名Top10'),
                     visualmap_opts=opts.VisualMapOpts(max_=1000)) 

In [19]:
#各省份螺蛳粉销售收入分布图
#结论：从地图中可以看出，广西、浙江、河南的销售收入分别位居前三位。其中广西的销售收入为1.9亿+元，是浙江销售收入的6倍左右。还有几个省份没有销售收入数据。

from pyecharts.charts import Map 
from pyecharts import options as opts

# 计算各省销售收入
province_sales = data.groupby('province')['sales'].sum().round()

# 绘制地图
map1 = Map(init_opts=opts.InitOpts(width='1350px', height='750px'))
map1.add("", [list(z) for z in zip(province_sales.index.tolist(), province_sales.values.tolist())],
         maptype='china'
        ) 
map1.set_global_opts(title_opts=opts.TitleOpts(title='国内各省份螺蛳粉销售收入分布'),
                     visualmap_opts=opts.VisualMapOpts(max_=100000000),
                     toolbox_opts=opts.ToolboxOpts()
                    ) 

In [20]:
#各省份螺蛳粉销量分布图
#结论：从地图中可以看出，广西、浙江、河南的销量分别位居前三位。其中广西的销量为447万+人次，是浙江销量的6倍左右。将销售收入与销量比较分析来看，广西省每人次平均消费42元。还有几个省份没有销量数据。

# 计算各省销量
province_num = data.groupby('province')['purchase_num_new'].sum()

# 绘制地图
map2 = Map(init_opts=opts.InitOpts(width='1350px', height='750px'))
map2.add("", [*zip(province_num.index, province_num.values)],
         maptype='china'
        ) 
map2.set_global_opts(title_opts=opts.TitleOpts(title='国内各省份螺蛳粉销量分布'),
                     visualmap_opts=opts.VisualMapOpts(max_=4000000),
                     toolbox_opts=opts.ToolboxOpts()
                    )

In [21]:
#广西省各市螺蛳粉销量分布图
#结论：从地图中可以看出，柳州、南宁、桂林的销量分别位居前三位，其中柳州的销量为423万+人次。而从各省份螺蛳粉销量分布图中我们知道广西省的总销量才447万+人次，说明广西省的销量基本来源于柳州市。还有几个城市没有销量数据。

# 计算广西省各市销量
city_num = data[data['province']=='广西'].groupby('city')['purchase_num_new'].sum()

# 绘制地图
map3 = Map(init_opts=opts.InitOpts(width='1350px', height='750px'))
map3.add("", [list(z) for z in zip((city_num.index+'市').tolist(), city_num.values.tolist())],
         maptype='广西'
        ) 
map3.set_global_opts(title_opts=opts.TitleOpts(title='广西省各市螺蛳粉销量分布'),
                     visualmap_opts=opts.VisualMapOpts(max_=500000),
                     toolbox_opts=opts.ToolboxOpts()
                    )

In [22]:
# 生成tab
from pyecharts.charts import Tab

tab = Tab()
tab.add(word1, '词云图')
tab.add(pie1, '饼图1')
tab.add(pie2, '饼图2')
tab.add(pie3, '饼图3')
tab.add(bar1, '条形图1')
tab.add(bar2, '条形图2')
tab.add(map1, '全国地图1')
tab.add(map2, '全国地图2')
tab.add(map3, '广西省地图')
tab.render('螺蛳粉销量分析可视化.html')

'D:\\Code\\Python\\螺蛳粉销量分析可视化.html'